In [ ]:
import os

download_name = "frankenstein_with_splits.csv.bz2"
if not os.path.exists(download_name):
    import requests
    response = requests.get(f"https://raw.githubusercontent.com/bzitko/nlp_repo/main/assignments/a04/{download_name}")
    with open(download_name, "wb") as fp:
        fp.write(response.content)
    response.close()

name = "frankenstein_with_splits.csv"
if not os.path.exists(name):
    import bz2
    with open(download_name, 'rb') as bzf, open(name, 'wb') as fp:
        fp.write(bz2.decompress(bzf.read()))        

# SOLUTION: Learning Embeddings with SkipGram

## Imports

In [1]:
import os
from argparse import Namespace
from collections import Counter
import json

import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

## Data Vectorization classes

### The Vocabulary

In [1]:
class Vocabulary(object):
    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
        
        self._add_unk = add_unk
        self._unk_token = unk_token
        
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token) 
        
    def to_serializable(self):
        return {'token_to_idx': self._token_to_idx, 
                'add_unk': self._add_unk, 
                'unk_token': self._unk_token}

    @classmethod
    def from_serializable(cls, contents):
        return cls(**contents)

    def add_token(self, token):
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
            
    def add_many(self, tokens):
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

    def lookup_index(self, index):
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    @classmethod
    def from_dataframe(cls, df):
        vocab = Vocabulary()
        for index, row in df.iterrows():
            vocab.add_token(row.context)
            vocab.add_token(row.target)            
        return vocab

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)



In [2]:
class Unigram(object):

    def __init__(self, freqs=None):
        if not freqs:
            freqs = {}
        self._freqs = freqs
        self._count = sum(self._freqs.values()) if self._freqs else 0
    
    def add_index(self, index):
        self._freqs[index] = self._freqs.get(i, 0) + 1
    
    def get_freq(self, index):
        return self._freqs[index]

    def get_prob(self, index):
        return self._freqs[index] / self._count

    def __len__(self):
        return len(self._freqs())


### The Dataset

In [45]:
class SkipGramDataset(Dataset):
    def __init__(self, df, vocab, unigram):
        self.df = df
        self._vocab = vocab
        self._unigram = unigram
                
        self._lookup_dict = {'train': self.df[self.df.split == "train"],
                             'val':   self.df[self.df.split == "val"],
                             'test':  self.df[self.df.split == "test"],}

        self.set_split('train')

    @classmethod
    def load_dataset_and_make_vocab(cls, csv):
        df = pd.read_csv(csv)
        return cls(df, Vocabulary.from_dataframe(df))

    def get_vocab(self):
        return self._vocab

    def get_unigram(self):
        return self._unigram
        
    def set_split(self, split="train"):
        self._target_split = split
        self._target_df = self._lookup_dict[split]

    def __len__(self):
        return len(self._target_df)

    def __getitem__(self, index):
        row = self._target_df.iloc[index]

        context_index = self._vocab.lookup_token(row.context)
        target_index = self._vocab.lookup_token(row.target)

        

        return {'x_data':   context_index,
                'y_target': target_index}

    def get_num_batches(self, batch_size):
        return len(self) // batch_size
    
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"): 
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

## The Model: SkipGram

#thumbsup

In [105]:
class SkipGramClassifier(nn.Module): # Simplified SkipGram Model
    def __init__(self, vocabulary_size, embedding_size):
        super(SkipGramClassifier, self).__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.embeddings_in =  nn.Embedding(num_embeddings=vocabulary_size, 
                                           embedding_dim=embedding_size)
        self.embeddings_out =  nn.Embedding(num_embeddings=vocabulary_size, 
                                            embedding_dim=embedding_size)                                           

    def forward(self, x_in):
        batch_size = x_in.shape[0]
        assert x_in.shape == (batch_size, )
        
        e_c = self.embeddings_in(x_in)
        assert e_c.shape == (batch_size, self.embedding_size)

        all_e_w = self.embeddings_out(torch.tensor([range(self.vocabulary_size)])).squeeze()
        assert all_e_w.shape == (self.vocabulary_size, self.embedding_size)
        
        all_sim_w_with_c = torch.matmul(all_e_w, e_c.T).T
        assert all_sim_w_with_c.shape == (batch_size, self.vocabulary_size)
        
        y_hat = F.log_softmax(all_sim_w_with_c, dim=1)
        assert y_hat.shape == (batch_size, self.vocabulary_size)

        return y_hat


loss_func = torch.nn.NLLLoss()
classifier = SkipGramClassifier(len(dataset._vocab), embedding_size=args.embedding_size)
for batch in generate_batches(dataset, 3):
    x = batch["x_data"]
    y = batch["y_target"]
    y_hat = classifier(x)
    loss = loss_func(y_hat, y)
    print(loss.item())
    
    break

40.27476501464844


## Training Routine

### Helper functions

In [106]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

#### general utilities

In [107]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

### Settings and some prep work

In [108]:
args = Namespace(
    # Data and Path information
    skipgram_csv="frankenstein_with_splits.csv",
    vectorizer_file="vectorizer_a04.1_skipgram.json",
    model_state_file="model_a04.1_skipgram.pth",
    save_dir=".",
    # Model hyper parameters
    embedding_size=50,
    # Training hyper parameters
    seed=1337,
    num_epochs=100,
    learning_rate=0.0001,
    batch_size=32,
    early_stopping_criteria=5,
    # Runtime options
    cuda=True,
    catch_keyboard_interrupt=True,
    reload_from_files=False,
    expand_filepaths_to_save_dir=True
)

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)
    
    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))
    

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
    
print("Using CUDA: {}".format(args.cuda))


# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# handle dirs
handle_dirs(args.save_dir)

Expanded filepaths: 
	./vectorizer_a04.1_skipgram.json
	./model_a04.1_skipgram.pth
Using CUDA: False


### Initializations

In [109]:
if args.reload_from_files:
    print("Loading dataset and loading vocabulary")
    dataset = SkipGramDataset.load_dataset_and_load_vectorizer(args.skipgram_csv,
                                                               args.vectorizer_file)
else:
    print("Loading dataset and creating vocabulary")
    dataset = SkipGramDataset.load_dataset_and_make_vocab(args.skipgram_csv)
    #dataset.save_vectorizer(args.vectorizer_file)
    
vocab = dataset.get_vocab()
print(vocab)

classifier = SkipGramClassifier(vocabulary_size=len(vocab), 
                                embedding_size=args.embedding_size)


Loading dataset and creating vocabulary
<Vocabulary(size=96668)>


### Training loop

In [110]:
classifier = classifier.to(args.device)
    
loss_func = nn.NLLLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.5,
                                                 patience=1)
train_state = make_train_state(args)

epoch_bar = tqdm(desc='training routine', 
                 total=args.num_epochs,
                 position=0)

dataset.set_split('train')
train_bar = tqdm(desc='split=train',
                 total=dataset.get_num_batches(args.batch_size),  
                 position=1, 
                 leave=True)
dataset.set_split('val')
val_bar = tqdm(desc='split=val',
               total=dataset.get_num_batches(args.batch_size), 
               position=1, 
               leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on

        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = classifier(x_in=batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss, acc=running_acc, 
                            epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # compute the output
            y_pred =  classifier(x_in=batch_dict['x_data'])

            # step 3. compute the loss)
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            val_bar.set_postfix(loss=running_loss, acc=running_acc, 
                                epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        if train_state['stop_early']:
            break

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
except KeyboardInterrupt:
    print("Exiting loop")


training routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/1984 [00:00<?, ?it/s]

split=val:   0%|          | 0/425 [00:00<?, ?it/s]

Exiting loop


In [111]:
# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename']))
classifier = classifier.to(args.device)
loss_func = nn.CrossEntropyLoss()

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(x_in=batch_dict['x_data'])
    
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc


In [112]:
print("Test loss: {};".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))

Test loss: 31.13783382640164;
Test Accuracy: 0.0


### Trained Embeddings

In [121]:
def pretty_print(results):
    """
    Pretty print embedding results.
    """
    for item in results:
        print ("...[%.2f] - %s"%(item[1], item[0]))

def get_closest(target_word, word_to_idx, embeddings, n=5):
    """
    Get the n closest
    words to your word.
    """

    # Calculate distances to all other words
    
    word_embedding = embeddings[word_to_idx[target_word.lower()]]
    distances = []
    for word, index in word_to_idx.items():
        if word == target_word:
            continue
        distances.append((word, torch.dist(word_embedding, embeddings[index])))
    
    results = sorted(distances, key=lambda x: x[1])[1:n+2]
    return results


In [122]:
word = "good"
embeddings = classifier.embeddings_in.weight.data
word_to_idx = dataset._vocab._token_to_idx
pretty_print(get_closest(word, word_to_idx, embeddings, n=5))

...[6.56] - own sake .
...[6.59] - hearing this he satisfied and consented
...[6.75] - and give him notice if any
...[6.75] - me to forget passing cares of
...[6.84] - exceeded theirs .
...[6.84] - been my laboratory 


In [124]:
target_words = ['frankenstein', 'monster', 'science', 'sickness', 'lonely', 'happy']

embeddings = classifier.embeddings_in.weight.data
word_to_idx = dataset._vocab._token_to_idx

for target_word in target_words: 
    print(f"======={target_word}=======")
    if target_word not in word_to_idx:
        print("Not in vocabulary")
        continue
    pretty_print(get_closest(target_word, word_to_idx, embeddings, n=5))

=======frankenstein=======
...[6.91] - i have visited lakes of lucerne
...[6.96] - few years ago loved and beloved
...[6.98] - , i , niece , and
...[6.98] - appears to be purport of my
...[7.04] - revisit my mind that i might
...[7.13] - the tenets of religion and taught
=======monster=======
...[6.47] - but it a still greater
...[6.59] - of the monstrous which i had
...[6.61] - and must feel 
...[6.64] - clerval was forever me , ghastly
...[6.65] - much more i myself was yet
...[6.74] - ardently desired the of knowledge .
=======science=======
...[6.81] - i feared to from the sight
...[6.88] - can conceive the i suffered during
...[6.91] - this time we to our house
...[7.05] - the passage towards south became perfectly
...[7.10] - effort destroyed all remaining strength i
...[7.11] - floating
=======sickness=======
...[6.82] - refreshed me and me with such
...[6.83] - , which every and then would
...[6.87] - the strange chances have lately occurred
...[6.93] - musty
...[7.02] - i than